In [36]:
import pickle
import re
from random import seed
from random import randint
import pymorphy2

In [37]:
with open('data.pickle',"rb") as file:
    synonDict = pickle.load(file)

**camel case - только для классов. для переменных и функций не используется!!!** заменить всякие getNormalSynonym на get_normal_synonym

лучше сделай это все в виде класса, потому что у тебя там достаточно много параметров, которые можно в конструктор запихать

вообще попробуй написать так, чтобы было меньше букав

In [42]:
#returns (set,info) - коммент не несет в себе полезной информации. Если уж так пишешь, то надо пояснять, что вообще за множество, какие его элементы и тд.
def getSetOfFeatursAndWordInfo(word):
    #подбор информации о слове в тексте - из коммента непонятно, что за информация
    morph = pymorphy2.MorphAnalyzer() # это бы я запихала в конструктор
    wordinfo = morph.parse(word)[0]
   
    features = [wordinfo.tag.case,
                wordinfo.tag.number,
                wordinfo.tag.gender,
                wordinfo.tag.tense]

    setOfNotNone = set()
    for feature in features:
        if feature != None:
            setOfNotNone.add(feature)
    return setOfNotNone,wordinfo


#returns sinonim word or sentence
def getNormalSynonym(word): #а что, кстати, значит "нормальный синоним"? :)
    morph = pymorphy2.MorphAnalyzer()
    #подбор информации о слове в тексте          
    
    setOfNotNone,wordinfo = getSetOfFeatursAndWordInfo(word)

    clearword    = wordinfo.normal_form #не надо несколько "равно" на одном уровне
    synonymslist = synonDict.get(clearword)
    if synonymslist != None:

        #взятие рандомного синонима из списка синонимов и обработка
        #(для изменения его в нужную фоорму и замену в тексте)
        synonymSentence = synonymslist[randint(0,len(synonymslist)-1)]
        synonymSentence = re.sub(r'\W|  |\n'," ",synonymSentence) #что таит в себе эта загадочная штука?
        words  = synonymSentence.split(" ")
        
        #если синоним для выбранного слова - предложение,то лучше его не менять - почему?
        if len(words) > 1:
            return synonymSentence

        #если слов нет, то не на что менять выбранное слово - тут понятно и без коммента
        if len(words) == 0:        
            return None
        synonym = words[0]
        #если слово пустое значит в словаре синонимов ничего не нашлось, значит ничго менять не нужно 
        if len(synonym) == 0:
            return None

        #синоним переведенный в нужную форму(если она существует)
        synonymMorphed = morph.parse(synonym)[0].inflect(setOfNotNone)
        
        # если есть информация о выбранном слове (чтобы можно было использовать метод inflect)
        # И слово видоизменяется (существует нужная форма слова)
        if len(setOfNotNone) != 0 and synonymMorphed is not None:
            #измененное слово в нужную форму, если можно
            synonym = synonymMorphed.word 

        return synonym

#returns sentence - снова коммент из разряда ни о чем
def synonymAugSentence(sentence,mode = "rand"):
    morph   = pymorphy2.MorphAnalyzer()
    outText = sentence #нет смысла создавать новую переменную из уже имеющейся, это только путает
    newText = re.sub(r'\W',' ',outText) #не совсем понятно, для каких случаев это надо
    words   = newText.split(" ")
    seed(version = 2)
    if mode == "all": #я бы вообще убрала этот параметр, а лучше бы сделала другой, который бы отвечал за количество слов, которые мы будем заменять в предложении на синонимы. Совсем замечательно, если это будет определяться через рандомайзер
        for wordInText in words:
            synonym = getNormalSinonim(wordInText)
            if sinonim is None:
                continue
            outText = re.sub(wordInText,synonym,outText)
                              
    if mode == "rand":
        indx = randint(0,len(words)-1)
        word =  words[indx]
        synonym = getNormalSynonym(word)
        if synonym is None:
            return outText
        outText = re.sub(word,synonym,outText) #есть более понятная функция replace для строки
        if indx > 0: #то, что начинается здесь и идет до конца, я вообще не особо поняла, нужно как-то понятней
            lastWord                  = words[indx-1] #не надо выравнивать несколько "равно", должно быть по одному пробелу с каждой стороны
            lastWordinfo              = morph.parse(lastWord)[0]
            setOfFeatursWord,wordinfo = getSetOfFeatursAndWordInfo(synonym)
            if lastWordinfo is not None:
                if wordinfo is not None:
                    normlastword = lastWordinfo.inflect(setOfFeatursWord)
                    if normlastword is not None:      
                        outText = re.sub(lastWord,normlastword.word,outText)

    return outText

In [70]:
def getAugdf(df,columnWithText):
    newdf = df.head
    
    

In [62]:
import pandas as pd
df_pos = pd.read_csv('positive.csv', sep=';', header=None).assign(positive=1)
df_neg = pd.read_csv('negative.csv', sep=';', header=None).assign(positive=0)

In [73]:
df_neg = df_neg.head(20)
df_neg

,0,1,2,3,4,5,6,7,8,9,10,11,positive
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,0
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,0
3,408906914437685248,1386325980,Poliwake,"Желаю хорошего полёта и удачной посадки,я буду...",-1,0,0,0,10628,207,200,0,0
4,408906914723295232,1386325980,capyvixowe,"Обновил за каким-то лешим surf, теперь не рабо...",-1,0,0,0,35,17,34,0,0
5,408906915704737792,1386325980,nunejibaduq,"Котёнка вчера носик разбила, плакала и расстра...",-1,0,0,0,222,62,62,0,0
6,408906973623504896,1386325994,Donna_132,"@juliamayko @O_nika55 @and_Possum Зашли, а то ...",-1,0,0,0,8311,36,68,0,0
7,408907184160776192,1386326044,Darynka_,а вообще я не болею - я не выздоравливаю :(,-1,0,0,0,7,43,33,1,0
8,408907406652231680,1386326097,mejagurijope,я микрофраза :( учимся срать кирпичами в режим...,-1,0,0,0,8,5,3,0,0
9,408907447303413760,1386326107,Ira_frank,"я хочу с тобой помириться , но сука я гордая и...",-1,0,0,0,5475,244,95,1,0


In [74]:
df_pos = df_pos.head(30)